In [10]:
# 240923 SLSQP case2
# SLSQP case2 最优值，123.33 找不到比SR更大的最优值，用DE结果代替

from scipy.optimize import minimize, differential_evolution, dual_annealing
import numpy as np
from floris.tools import FlorisInterface
import time


# get layout case 2
file_path = r'C:\Users\tuyu0\Desktop\Apaper3\case2\layout.txt'  # Replace with your actual file path
# 读取文件内容
with open(file_path, 'r') as file:
    lines = file.readlines()
# 初始化空列表
layout_x = []
layout_y = []
# 处理每一行
for line in lines:
    # 去除行首的 "- [" 和行尾的 "]"
    line = line.strip().strip('- [').strip(']')
    # 拆分成两个数值
    x, y = map(float, line.split(','))
    # 添加到列表中
    layout_x.append(x)
    layout_y.append(y)
layout_x = np.array(layout_x)
layout_y = np.array(layout_y)

  
def get_floris_power_gch60(X_yaw):
    # change the x into integer type
    X_yaw = np.round(X_yaw).astype(int)
    # 第一个代码段中的初始化风场模型
    wakemodel = 'gch'
    wakemodelpath = 'D:/AkeyanCode/florisv3.4/examples/inputs/' + wakemodel + '.yaml'
    fi = FlorisInterface(wakemodelpath)

    fi.reinitialize(
        layout_x=layout_x,
        layout_y=layout_y,
        air_density = 1.225,
        wind_directions=[234.0], 
        wind_speeds=[8.11],
        wind_shear=0.15,
        turbulence_intensity=0.075,
    )

    yaw_angles = np.zeros([1, 1, len(layout_x)])

    yaw_angles[0, 0, :] = X_yaw  # 直接替换整个最后一维

    fi.calculate_wake(yaw_angles=yaw_angles)
    turbine_powers = fi.get_turbine_powers().flatten() / 1e6
    total = np.sum(turbine_powers)

    return -total

def objective_function(x):
    return get_floris_power_gch60(np.array(x).reshape(1,-1))

# 定义变量维度
num_dimensions = len(layout_x)
# 每个维度的变化范围
range_per_dimension = 25
# 生成范围数组
bounds = np.array([[-range_per_dimension, range_per_dimension]] * num_dimensions)

# 设置优化参数
options = {'maxiter': 200, 'ftol': 1e-3, 'eps': 0.02}
# options = {'maxiter': 200, 'ftol': 1e-13}
# 默认情况下，'eps'参数的值为 1.4901161193847656e-08


# 运行次数
n_run = 30 #暂时25次，找到最高功率对应的偏航角即可

# 用于存储每次运行的结果
Allfun = []
Alltime = []
Allnfev = []
angles = []
# 运行多次优化 n_run
for i in range(n_run):
    print('n_run',i)
    # 随机生成起始点
    x_start = np.random.uniform(-range_per_dimension, range_per_dimension, size=(num_dimensions,))
    
    start_time = time.time()
    
    result = minimize(
    objective_function,
    x_start,
    method="SLSQP",
    bounds=bounds,
    options=options,
    )
    
    execution_time = time.time() - start_time
    Allfun.append(result.fun)
    Alltime.append(execution_time)
    Allnfev.append(result.nfev)
    angles.append(result.x)  # 收集偏航角
    



n_run 0
n_run 1
n_run 2
n_run 3
n_run 4
n_run 5
n_run 6
n_run 7
n_run 8
n_run 9
n_run 10
n_run 11
n_run 12
n_run 13
n_run 14
n_run 15
n_run 16
n_run 17
n_run 18
n_run 19
n_run 20
n_run 21
n_run 22
n_run 23
n_run 24
n_run 25
n_run 26
n_run 27
n_run 28
n_run 29


In [13]:
# SLSQP 用到的minimize代码，自变量不是整数，而是浮点数类型，因此需要修改 objective function
# 得到 maxAngle = np.round(maxAngle).astype(int) 后，验证功率和偏航角是否对应
import numpy as np

powerSR = - 129.24

# 找出小于 powerSR 的数值及其索引
below_powerSR = [(i, val) for i, val in enumerate(Allfun) if val < powerSR]

# 打印小于 powerSR 的数值及其位置
print("小于 powerSR 的数值及其位置:")
for index, value in below_powerSR:
    print(f"位置: {index}, 数值: {value}")

# 找到最小值及其位置
min_value = min(Allfun)
min_index = Allfun.index(min_value)

# 打印最小值及其位置
print(f"最小值: {min_value}, 位置: {min_index}")

print('Allfun',Allfun[min_index])
# print('Allnfev',Allnfev[min_index])
# print('Alltime',Alltime[min_index])
print('angles',angles[min_index])

maxAngle = angles[min_index]
maxAngle = np.round(maxAngle).astype(int)
print('maxAngle',maxAngle.shape)
# powerMax = get_floris_power_gch60(maxAngle)

print('##########################################################################')

# 第一个代码段中的初始化风场模型
wakemodel = 'gch'
wakemodelpath = 'D:/AkeyanCode/florisv3.4/examples/inputs/' + wakemodel + '.yaml'
fi = FlorisInterface(wakemodelpath)

fi.reinitialize(
    layout_x=layout_x,
    layout_y=layout_y,
    air_density = 1.225,
    wind_directions=[234.0], 
    wind_speeds=[8.11],
    wind_shear=0.15,
    turbulence_intensity=0.075,
)

yaw_angles = np.zeros([1, 1, len(layout_x)])
print('yaw_angles',yaw_angles.shape)
yaw_angles[0, 0, :] = maxAngle  # 直接替换整个最后一维
print('yaw_angles',yaw_angles)

fi.calculate_wake(yaw_angles=yaw_angles)
turbine_powers = fi.get_turbine_powers().flatten() / 1e6
total = np.sum(turbine_powers)
print('total',total)

# SLSQP case2 最优值，123.33 找不到比SR更大的最优值，用DE结果代替

小于 powerSR 的数值及其位置:
最小值: -123.33221967880431, 位置: 28
Allfun -123.33221967880431
angles [ 15.3346728  -11.21016178  14.30089352 -15.37777008  23.65879292
   5.60055601   3.82937325 -17.47763103  24.01673177  24.39771567
  -3.77629941  -6.22152366 -23.39871478   7.81975052 -23.74715527
  -5.262091    -3.44840189  12.17163949 -19.00153029   7.6764329
 -14.49144563  -3.05875833  -8.09899137  -8.27053136  -5.35495866
  10.15186237  21.76538247  10.33672266  16.35340044   9.87120479
  -0.31434885  17.11608994 -11.276247    11.63548072  18.45610336
  21.91332804 -13.42341377 -13.88151484 -11.80066571  22.73275611
   4.95429926 -15.82846683  21.39159217  18.64551157  -5.19149938
 -23.42121385   6.44496071  -1.31821884  -1.254416     9.72580574
   4.09740682  17.11206693  15.1219254   17.91803367   8.88587484
 -11.18524676   0.55125861 -19.21005566  -6.34121077  -6.12281039
  12.0166466  -10.40846887 -11.63056124 -12.40935758  -0.47315693
  -6.87664327 -19.06987386   0.7897143   14.33754055 -24

In [27]:
# 240923 CUE推荐全文, 运行SLSQP 得到最优偏航角
# Amalia wind farm, compare six methods
# smt 核

from scipy.optimize import minimize, differential_evolution, dual_annealing
import numpy as np
from floris.tools import FlorisInterface
import scipy.io as sio
import time

# get layout
file_path = r'D:\Akeyan\matlab\230424liumiantiWindFarm\0514windFarmPicture\Amalia_locAndHull.mat'
# 加载MAT文件
mat_data = sio.loadmat(file_path)
# 提取turbineX和turbineY数据
turbineX = mat_data['turbineX']
turbineY = mat_data['turbineY']
# 将坐标减去2000
turbineX_adjusted = turbineX - 2000
turbineY_adjusted = turbineY - 2000
# 转换为Python列表
layout_x = turbineX_adjusted.flatten().tolist()
layout_y = turbineY_adjusted.flatten().tolist()
print('length(layout_x)',len(layout_x))    
def get_floris_power_gch60(X_yaw):
    # change the x into integer type
    X_yaw = np.round(X_yaw).astype(int)
    # 第一个代码段中的初始化风场模型
    wakemodel = 'gch'
    wakemodelpath = 'D:/AkeyanCode/florisv3.4/examples/inputs/' + wakemodel + '.yaml'
    fi = FlorisInterface(wakemodelpath)

    fi.reinitialize(
        layout_x=layout_x,
        layout_y=layout_y,
        air_density = 1.23,
        wind_directions=[220.0], 
        wind_speeds=[11.3],
        wind_shear=0.1,
        turbulence_intensity=0.06,
    )

    yaw_angles = np.zeros([1, 1, len(layout_x)])

    yaw_angles[:, :, :X_yaw.shape[1]] = X_yaw
    # print('yaw_angles',yaw_angles.shape,yaw_angles)
    # In SLSQP, is X_yaw.shape[1], not [0] 
    fi.calculate_wake(yaw_angles=yaw_angles)
    turbine_powers = fi.get_turbine_powers().flatten() / 1e6
    total = np.sum(turbine_powers)
    # total is a 'list' object, but should be Python 浮点数,一个浮点数标量，形状为()
    # print('total',total)
    # print(type(total))
    return -total

def objective_function(x):
    return get_floris_power_gch60(np.array(x).reshape(1,-1))

# 定义变量维度
num_dimensions = len(layout_x)
# 每个维度的变化范围
range_per_dimension = 25
# 生成范围数组
bounds = np.array([[-range_per_dimension, range_per_dimension]] * num_dimensions)

# 设置优化参数
options = {'maxiter': 200, 'ftol': 1e-3, 'eps': 0.02}
# options = {'maxiter': 200, 'ftol': 1e-13}
# 默认情况下，'eps'参数的值为 1.4901161193847656e-08


# 运行次数
n_run = 25 #暂时25次，找到最高功率对应的偏航角即可

# 用于存储每次运行的结果
Allfun = []
Alltime = []
Allnfev = []
angles = []
# 运行多次优化 n_run
for i in range(n_run):
    print('n_run',i)
    # 随机生成起始点
    x_start = np.random.uniform(-range_per_dimension, range_per_dimension, size=(num_dimensions,))
    
    start_time = time.time()
    
    result = minimize(
    objective_function,
    x_start,
    method="SLSQP",
    bounds=bounds,
    options=options,
    )
    
    execution_time = time.time() - start_time
    Allfun.append(result.fun)
    Alltime.append(execution_time)
    Allnfev.append(result.nfev)
    angles.append(result.x)  # 收集偏航角

# print('powerBase', get_floris_power_gch60(np.zeros(len(layout_x))))
# powerBase -232.1665

length(layout_x) 60
n_run 0
n_run 1
n_run 2
n_run 3
n_run 4
n_run 5
n_run 6
n_run 7
n_run 8
n_run 9
n_run 10
n_run 11
n_run 12
n_run 13
n_run 14
n_run 15
n_run 16
n_run 17
n_run 18
n_run 19
n_run 20
n_run 21
n_run 22
n_run 23
n_run 24


In [31]:
# SLSQP 用到的minimize代码，自变量不是整数，而是浮点数类型，因此需要修改 objective function
# 得到 maxAngle = np.round(maxAngle).astype(int) 后，验证功率和偏航角是否对应
import numpy as np

powerSR = - 238.11

# 找出小于 powerSR 的数值及其索引
below_powerSR = [(i, val) for i, val in enumerate(Allfun) if val < powerSR]

# 打印小于 powerSR 的数值及其位置
print("小于 powerSR 的数值及其位置:")
for index, value in below_powerSR:
    print(f"位置: {index}, 数值: {value}")

# 找到最小值及其位置
min_value = min(Allfun)
min_index = Allfun.index(min_value)

# 打印最小值及其位置
print(f"最小值: {min_value}, 位置: {min_index}")

print('Allfun',Allfun[min_index])
print('Allnfev',Allnfev[min_index])
print('Alltime',Alltime[min_index])
print('angles',angles[min_index])

maxAngle = angles[min_index]
maxAngle = np.round(maxAngle).astype(int)
print('maxAngle',maxAngle.shape)
# powerMax = get_floris_power_gch60(maxAngle)

##########################################################################3

# 第一个代码段中的初始化风场模型
wakemodel = 'gch'
wakemodelpath = 'D:/AkeyanCode/florisv3.4/examples/inputs/' + wakemodel + '.yaml'
fi = FlorisInterface(wakemodelpath)

fi.reinitialize(
    layout_x=layout_x,
    layout_y=layout_y,
    air_density = 1.23,
    wind_directions=[220.0], 
    wind_speeds=[11.3],
    wind_shear=0.1,
    turbulence_intensity=0.06,
)

yaw_angles = np.zeros([1, 1, len(layout_x)])
print('yaw_angles',yaw_angles.shape)
yaw_angles[0, 0, :] = maxAngle  # 直接替换整个最后一维
print('yaw_angles',yaw_angles)

# print('yaw_angles',yaw_angles.shape,yaw_angles)
# In SLSQP, is X_yaw.shape[1], not [0] 
fi.calculate_wake(yaw_angles=yaw_angles)
turbine_powers = fi.get_turbine_powers().flatten() / 1e6
total = np.sum(turbine_powers)
# total is a 'list' object, but should be Python 浮点数,一个浮点数标量，形状为()
# print('total',total)
# print(type(total))
print('total',total)

# 240923 SLSQP 最佳功率对应的偏航角，整数
# yaw_angles [[[ 10.  12.  -1.  21.  24.  -5. -11.  15.  12.  12.   0. -17. -21.  12.
#     18.   7.  -4. -16. -23.   9.  -4.  -7. -25.  -2.  25.  13.  19.   4.
#    -12. -13. -13. -17.  -1. -14.  -9. -10.  14.  -6. -13. -23.   0. -17.
#     10. -14.  13.  -2.  -3. -25.  -4. -20.  14.  -7. -14.  19.  -4. -12.
#      1.   4. -17.   7.]]]
# total 228.32571177353168


小于 powerSR 的数值及其位置:
最小值: -238.03850492966083, 位置: 0
Allfun -238.03850492966083
Allnfev 2200
Alltime 704.6392953395844
angles [17, 15, 13, 20, 15, 10, 10, 1, 18, 14, 12, 10, -1, -2, 19, 14, 11, 9, 7, 2, 1, 16, 14, 11, 5, -1, -3, 15, 12, 7, 7, -8, 0, -1, 13, -15, -16, 6, 0, 4, -1, -17, -18, 1, 2, -2, 0, 1, -15, -15, -1, -1, 0, 1, -1, 3, -2, -2, 0, 0]
maxAngle (60,)
yaw_angles (1, 1, 60)
yaw_angles [[[ 17.  15.  13.  20.  15.  10.  10.   1.  18.  14.  12.  10.  -1.  -2.
    19.  14.  11.   9.   7.   2.   1.  16.  14.  11.   5.  -1.  -3.  15.
    12.   7.   7.  -8.   0.  -1.  13. -15. -16.   6.   0.   4.  -1. -17.
   -18.   1.   2.  -2.   0.   1. -15. -15.  -1.  -1.   0.   1.  -1.   3.
    -2.  -2.   0.   0.]]]
total 238.03850492966083


In [29]:
# 240923 CUE推荐全文, 运行PSO 得到最优偏航角
# Amalia wind farm, compare six methods
# smt 核

import numpy as np
from floris.tools import FlorisInterface
import scipy.io as sio
import time
import random


# get layout
file_path = r'D:\Akeyan\matlab\230424liumiantiWindFarm\0514windFarmPicture\Amalia_locAndHull.mat'
# 加载MAT文件
mat_data = sio.loadmat(file_path)
# 提取turbineX和turbineY数据
turbineX = mat_data['turbineX']
turbineY = mat_data['turbineY']
# 将坐标减去2000
turbineX_adjusted = turbineX - 2000
turbineY_adjusted = turbineY - 2000
# 转换为Python列表
layout_x = turbineX_adjusted.flatten().tolist()
layout_y = turbineY_adjusted.flatten().tolist()

def get_floris_power_gch60(X_yaw):
    
    # 第一个代码段中的初始化风场模型
    wakemodel = 'gch'
    wakemodelpath = 'D:/AkeyanCode/florisv3.4/examples/inputs/' + wakemodel + '.yaml'
    fi = FlorisInterface(wakemodelpath)

    fi.reinitialize(
        layout_x=layout_x,
        layout_y=layout_y,
        air_density = 1.23,
        wind_directions=[220.0], 
        wind_speeds=[11.3],
        wind_shear=0.1,
        turbulence_intensity=0.06,
    )

    yaw_angles = np.zeros([1, 1, len(layout_x)])
    # print('X_yaw',X_yaw.shape, X_yaw) # X_yaw.shape = (60,)
    yaw_angles[0, 0, :] = X_yaw  # 直接替换整个最后一维

    fi.calculate_wake(yaw_angles=yaw_angles)
    turbine_powers = fi.get_turbine_powers().flatten() / 1e6
    total = np.sum(turbine_powers)
    return -total

def objective_function(x):
    return get_floris_power_gch60(np.array(x).reshape(1,-1))

def pso(objective_function, bounds, info=True):
    dim = bounds.shape[0]
    lb = bounds[:, 0].tolist()
    ub = bounds[:, 1].tolist()
    # Initialization
    ratio1 = 0.5
    ratio2 = 0.5
    noP=20 # number of particles
    Max_iteration=200
    Vmax=6
    wMax=0.9
    wMin=0.2
    c1=2
    c2=2
    class Particle:
        def __init__(self, dim, lb, ub):
            self.X = [random.randint(l, u) for l, u in zip(lb, ub)]
            self.PBEST_X = self.X.copy()
            self.PBEST_O = np.inf
            self.V = np.random.rand(dim)

    Swarm = []
    for _ in range(noP):
        Swarm.append(Particle(dim, lb, ub))
    # for particle in Swarm:
    #     print(f"Particle X: {particle.X}")
        
    # Global best initialization
    GBEST_X = np.zeros(dim)
    GBEST_O = np.inf

    # Variables for consecutive same best check
    previous_best_x = np.zeros(dim)
    consecutive_same_best_count = 0
    consecutive_same_best_limit = 5 # change

    for t in range(Max_iteration):
        # Calculate objective function and update personal best and global best
        for k in range(noP):
            Swarm[k].O = objective_function(Swarm[k].X)
            if Swarm[k].O < Swarm[k].PBEST_O:
                Swarm[k].PBEST_O = Swarm[k].O
                Swarm[k].PBEST_X = Swarm[k].X.copy()

            if Swarm[k].O < GBEST_O:
                GBEST_O = Swarm[k].O
                GBEST_X = Swarm[k].X.copy()
        # Check for consecutive same best if t > int(Max_iteration * ratio2)
        if t > int(Max_iteration * ratio2):
            GBEST_X = np.round(GBEST_X).astype(int)
            GBEST_O = objective_function(GBEST_X)
            if np.array_equal(GBEST_X, previous_best_x):
                consecutive_same_best_count += 1
            else:
                consecutive_same_best_count = 0

            # Update previous best
            previous_best_x = GBEST_X.copy()

            # Stop convergence if consecutive same best limit is reached
            if consecutive_same_best_count >= consecutive_same_best_limit:
                if info:
                    print(f"Convergence achieved: {consecutive_same_best_limit} consecutive same best.")
                break

        # Check for rounding to integers if t > int(Max_iteration * ratio)
        if t > int(Max_iteration * ratio1):
            for k in range(noP):
                Swarm[k].X = [int(x) for x in Swarm[k].X]
                

        # Update inertia weight
        w = wMax - t * ((wMax - wMin) / Max_iteration)

        # Update velocity and position
        for k in range(noP):
            # first_vel = Swarm[k].V.copy()
            Swarm[k].V = (w * np.array(Swarm[k].V) +
                c1 * np.random.rand(dim) * (np.array(Swarm[k].PBEST_X) - np.array(Swarm[k].X)) +
                c2 * np.random.rand(dim) * (GBEST_X - np.array(Swarm[k].X)))
    

            # Apply velocity limits
            Swarm[k].V[Swarm[k].V > Vmax] = Vmax * np.random.rand()
            Swarm[k].V[Swarm[k].V < -Vmax] = -Vmax * np.random.rand()

            # Update position and handle position limits
            # first_loc = Swarm[k].X.copy()
            Swarm[k].X = Swarm[k].X + Swarm[k].V

            # Handle position limits
            Swarm[k].X = [min(x, u) if x > u else x for x, u in zip(Swarm[k].X, ub)]
            Swarm[k].X = [max(x, l) if x < l else x for x, l in zip(Swarm[k].X, lb)]
        if info:
            print(f"Iteration {t + 1}, Current Best Objective Value: {GBEST_O} at {GBEST_X}")
    GBEST_Xint = np.round(GBEST_X).astype(int)
    GBEST_O = objective_function(GBEST_Xint)
    if info:
        print(f"Iteration {t + 1}, Current Best Objective Value Int: {GBEST_O} at {GBEST_Xint}")

    # Create a dictionary to store the results
    result = {
        'message': 'Optimization terminated successfully.',
        'success': True,
        'fun': GBEST_O,
        'x': GBEST_Xint.tolist(),
        'nit': t + 1,
        'nfev': (t + 1) * noP,  # Assuming each particle evaluation is counted
    }

    return result

####################################################################
# 定义变量维度
num_dimensions = len(layout_x)
# 每个维度的变化范围
range_per_dimension = 25
# 生成范围数组
bounds = np.array([[-range_per_dimension, range_per_dimension]] * num_dimensions)

# 运行次数
n_run = 25
# 用于存储每次运行的结果
Allfun = []
Allnfev = []
Alltime = []
angles = []
for i in range(n_run):
    print('run',i)
    start_time = time.time()
    result = pso(objective_function, bounds)
    end_time = time.time()
    execution_time = end_time - start_time
    
    # results.append(result)
    Allfun.append(result['fun'])
    Alltime.append(execution_time)
    Allnfev.append(result['nfev'])
    angles.append(result['x'])




run 0
Iteration 1, Current Best Objective Value: -228.93153647363775 at [16, 22, -12, -1, -2, 15, -3, 20, -14, 22, 5, 2, -15, -5, 9, -6, -19, -1, -11, 8, 13, 21, -17, -20, -21, 2, -6, 9, 0, 0, 1, -4, -4, -16, -20, 8, -10, 21, 14, 15, 9, -25, 7, 0, 2, 8, 4, 5, 5, -23, 6, -12, 24, 21, 1, -17, -17, -20, -17, -6]
Iteration 2, Current Best Objective Value: -229.46627300119945 at [-12.57349631234592, -11.57349631234592, -19.57349631234592, 10.112515904093268, 5.112515904093267, 15.853996540177674, 14.112515904093268, -19.9819360277739, 11.112515904093268, 18.42650368765408, 3.4730152106945003, 0.42650368765407887, -6.887484095906733, 8.055637420494461, -2.573496312345921, 6.112515904093267, -5.887484095906733, 16.112515904093268, -9.228369891770253, -4.573496312345921, -3.573496312345921, -3.573496312345921, 2.112515904093267, 2.112515904093267, 5.112515904093267, 2.2388234970457086, 4.112515904093267, 15.112515904093268, 2.112515904093267, 3.112515904093267, 15.112515904093268, -7.970667288

In [30]:
# PSO
# 得到 maxAngle = np.round(maxAngle).astype(int) 后，验证功率和偏航角是否对应
import numpy as np

powerSR = - 238.11

# 找出小于 powerSR 的数值及其索引
below_powerSR = [(i, val) for i, val in enumerate(Allfun) if val < powerSR]

# 打印小于 powerSR 的数值及其位置
print("小于 powerSR 的数值及其位置:")
for index, value in below_powerSR:
    print(f"位置: {index}, 数值: {value}")

# 找到最小值及其位置
min_value = min(Allfun)
min_index = Allfun.index(min_value)

# 打印最小值及其位置
print(f"最小值: {min_value}, 位置: {min_index}")

print('Allfun',Allfun[min_index])
print('Allnfev',Allnfev[min_index])
print('Alltime',Alltime[min_index])
print('angles',angles[min_index])

maxAngle = angles[min_index]
maxAngle = np.round(maxAngle).astype(int)
print('maxAngle',maxAngle.shape)
# powerMax = get_floris_power_gch60(maxAngle)

##########################################################################3

# # 第一个代码段中的初始化风场模型
# wakemodel = 'gch'
# wakemodelpath = 'D:/AkeyanCode/florisv3.4/examples/inputs/' + wakemodel + '.yaml'
# fi = FlorisInterface(wakemodelpath)

# fi.reinitialize(
#     layout_x=layout_x,
#     layout_y=layout_y,
#     air_density = 1.23,
#     wind_directions=[220.0], 
#     wind_speeds=[11.3],
#     wind_shear=0.1,
#     turbulence_intensity=0.06,
# )

# yaw_angles = np.zeros([1, 1, len(layout_x)])
# print('yaw_angles',yaw_angles.shape)
# yaw_angles[0, 0, :] = maxAngle  # 直接替换整个最后一维
# print('yaw_angles',yaw_angles)

# # print('yaw_angles',yaw_angles.shape,yaw_angles)
# # In SLSQP, is X_yaw.shape[1], not [0] 
# fi.calculate_wake(yaw_angles=yaw_angles)
# turbine_powers = fi.get_turbine_powers().flatten() / 1e6
# total = np.sum(turbine_powers)
# # total is a 'list' object, but should be Python 浮点数,一个浮点数标量，形状为()
# # print('total',total)
# # print(type(total))
# print('total',total)

# 240923 PSO 最佳功率对应的偏航角，整数
# angles [18, 13, 14, 18, 13, 10, 14, 1, 19, 13, 11, 12, 1, -2, 19, 14, 16, 8, 10, 0, 4, 13, 12, 15, 11, 0, 1, 15, 13, 9, 8, -5, 1, 0, -19, -15, -15, -7, 1, 3, 0, -18, -15, 4, 1, 2, 1, -1, -16, -14, 7, 1, 1, -1, -1, 0, -1, 5, 0, 2]
# total -238.00836

小于 powerSR 的数值及其位置:
最小值: -238.03850492966083, 位置: 0
Allfun -238.03850492966083
Allnfev 2200
Alltime 704.6392953395844
angles [17, 15, 13, 20, 15, 10, 10, 1, 18, 14, 12, 10, -1, -2, 19, 14, 11, 9, 7, 2, 1, 16, 14, 11, 5, -1, -3, 15, 12, 7, 7, -8, 0, -1, 13, -15, -16, 6, 0, 4, -1, -17, -18, 1, 2, -2, 0, 1, -15, -15, -1, -1, 0, 1, -1, 3, -2, -2, 0, 0]
maxAngle (60,)
